<a href="https://colab.research.google.com/github/pxyzP/phish-off-2.0/blob/main/phish_off_2_0_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phish-Off 2.0 !

**1. upload library ที่จำเป็น**

In [19]:
import pandas as pd
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd


**2. โหลด dataset จาก google drive**

In [20]:
from google.colab import drive
drive.mount("/content/gdrive")
urlsdata2 = pd.read_csv('/content/gdrive/My Drive/urlgoodbad2.0.1.csv')
urlsdata2.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,url,type
0,diaryofagameaddict.com,bad
1,diaryofagameaddict.com,bad
2,diaryofagameaddict.com,bad
3,diaryofagameaddict.com,bad
4,diaryofagameaddict.com,bad


**3. define dictionary เพื่อเปลี่ยน good --> safe, bad--> unsafe**

In [21]:
mapping2 = {
    'good': 'safe',
    'bad': 'unsafe'
}

urlsdata2 = urlsdata2.replace({'type': mapping2})
urlsdata2.to_csv('new_urlsdataset2.csv', index=False)

 4. สร้าง function makeTokensเพื่อ clean data โดยทำการแบ่ง token โดย / . - และลบ .com https http และ www ออก

In [22]:
def makeTokens(f):
    tkns_BySlash = f.split('/')
    total_Tokens = []

    for i in tkns_BySlash:
        tokens = i.split('-')
        tkns_ByDot = []

        for j in range(0, len(tokens)):
            temp_Tokens = tokens[j].split('.')
            tkns_ByDot = tkns_ByDot + temp_Tokens

        total_Tokens = total_Tokens + tokens + tkns_ByDot
        total_Tokens = list(set(total_Tokens))

    if 'com' in total_Tokens:
        total_Tokens.remove('com')
    if 'https:' in total_Tokens:
        total_Tokens.remove('https:')
    if 'http:' in total_Tokens:
        total_Tokens.remove('http:')
    if 'www' in total_Tokens:
        total_Tokens.remove('www')

    return total_Tokens


**5. แปลงแต่ละ column เป็น list**

In [23]:
url_list = urlsdata2["url"]
y = urlsdata2["type"]

**6. โหลด model Logistic regression  และ vectorizer และแบ่งข้อมูล train test แล้ว เริ่ม train ได้**

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer= makeTokens)
X = vectorizer.fit_transform(url_list)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [25]:
logit = LogisticRegression()
logit.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

**7. นำ test set มา test accuracy**

In [26]:
print("Accuracy ",logit.score(X_test, y_test))

Accuracy  0.9604737442922374


**8. save model และ vectorizer**

In [27]:
import pickle
with open("logit_model.pkl", "wb") as file:
    pickle.dump(logit, file)
with open("vectorizer.pkl", 'wb') as file:
    pickle.dump(vectorizer, file)
